

[Hakan CEBECİ'nin](http://https//www.udemy.com/user/software-development/)

* [Doğal Dil İşleme A-Z™: (NLP)](http://https//www.udemy.com/dogal-dil-isleme/)

kursundan öğrendiklerimi denediğim ve derlediğim kernelimdir.


In [ ]:
import numpy as np
import pandas as pd
#pandas data setı okumaya ve duzenlemeye yardımcı olur
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding, CuDNNGRU
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [ ]:
dataset = pd.read_csv('../input/hepsiburada.csv')
dataset.head()
#dataframe olrak yuklendı

In [ ]:
target = dataset['Rating'].values.tolist()
data = dataset['Review'].values.tolist()
cutoff = int(len(data) * 0.80)
x_train, x_test = data[:cutoff], data[cutoff:]
y_train, y_test = target[:cutoff], target[cutoff:]
print(x_train[100])
print(y_train[100])

In [ ]:
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)
#en sık gecen 10bın kelıme kelıme haznesınde
tokenizer.fit_on_texts(data)
#tokenlestırme yapıldı
#tokenizer.word_index

In [ ]:
x_train_tokens = tokenizer.texts_to_sequences(x_train)
#data ıcın yapılan durumun aynısı eğitim verisi içinde yapılıyor
print(x_train[800])
#secılen cumle asagıda
print(x_train_tokens[800])
#secılen cumlenın token halı asagıda

In [ ]:
x_test_tokens = tokenizer.texts_to_sequences(x_test)
#test ıcındekı cumleler tokenlestırılıyor
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)
# rnn belırlı boyutta verı verılmelı bu yuzden tum verılerı aynı boyuta getırmelıyız
# np array amac daha kolay ıslem yapmak
np.mean(num_tokens)
#ortalama 20 token var her cumlede

In [ ]:
print(np.max(num_tokens))
# en con 195 kelıme
print(np.argmax(num_tokens))
#en uzun olan 295 sırasını bulmak ıcın
print(x_train[21941])
#en uzuna bakalım

In [ ]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
print(max_tokens) #burada mean ve standart sapma degerlerıyle kac kelıme alsak daha ıyı olur onu secıyoruz
print(np.sum(num_tokens < max_tokens) / len(num_tokens))
#kac tane cumle 59 kaplıyor yaklasık yuzde 95

In [ ]:
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens)
#traın ıcın 59 dan sonrası sılıyorz
print(x_train_pad.shape)
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens)
#test ıcın 59 dan sonrası sılıyorz
print(x_test_pad.shape)

In [ ]:
print(np.array(x_train_tokens[800]))
print(x_train_pad[800])
# burada paddıng eklenmısler var

In [ ]:
idx = tokenizer.word_index
#kelımeler ve buna karsılık gelen degerler burada
inverse_map = dict(zip(idx.values(), idx.keys()))
#bu işlem ile hangı kelıme hangı token aldı bunu hesaplıyoruz
# cunku keras kelıme token yapıyo ama gerı cozemıyoruz bu yuzden onemlı

def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token!=0]
    text = ' '.join(words)
    return text
#buraya hangı token versek text halınde gerı donucek

In [ ]:
print(x_train[800])
#bu orjınal 800 ındextekı cumle
print(tokens_to_string(x_train_tokens[800]))

In [ ]:
model = Sequential()
embedding_size = 50
#her keılmeye karsılık gelen 50 uzunlugunda vektor var kelıme degerlının tutuldugu
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='embedding_layer'))
#ınput belırlenen maksımum deger kadar
#cıkıs 50 vektor 
#gırıs ıse verıler kelımeler (10bın uzunlugunda verı olucak)
model.add(GRU(units=16, return_sequences=True))
#return true olma sebebı tum cıkısların dıger aglara aktarılması, eger false olsaysı sadece son output cıkcaktı
#eger gpu olurda CuDNNGRU kullanılabılır, sadece nvdıa nın kartında calısyo ve sadece hız farkı var
model.add(GRU(units=8, return_sequences=True))
model.add(GRU(units=4)) #default false o yuzden yazmaya gerek yok
model.add(Dense(1, activation='sigmoid'))
optimizer = Adam(lr=1e-3)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(x_train_pad, y_train, epochs=5, batch_size=256)

In [ ]:
result = model.evaluate(x_test_pad, y_test)
print(result[1])

In [ ]:
y_pred = model.predict(x=x_test_pad[0:1000])
#y pred normalde sutun halınde
y_pred = y_pred.T[0] #burada transpozu alınarak yatay hale getırılıyor
cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in y_pred])
cls_true = np.array(y_test[0:1000])
incorrect = np.where(cls_pred != cls_true)
incorrect = incorrect[0] #tuple ıceısınde o yuzden [0]
print(incorrect) #yanlıs tahmınler
print(len(incorrect))  # yanlıs tahmın sayısı
idx = incorrect[3] #ılk yanlıs tahmın ındexi
print(idx)
text = x_test[idx]
print(text)
print("y_pred",y_pred[idx])
print("cls_pred",cls_pred[idx])
print("cls_true",cls_true[idx])

In [ ]:
text1 = "bu ürün çok iyi herkese tavsiye ederim"
text2 = "kargo çok hızlı aynı gün elime geçti"
text3 = "büyük bir hayal kırıklığı yaşadım bu ürün bu markaya yakışmamış"
text4 = "mükemmel"
text5 = "tasarımı harika ancak kargo çok geç geldi ve ürün açılmıştı tavsiye etmem"
text6 = "hiç resimde gösterildiği gibi değil"
text7 = "kötü yorumlar gözümü korkutmuştu ancak hiçbir sorun yaşamadım teşekkürler"
text8 = "hiç bu kadar kötü bir satıcıya denk gelmemiştim ürünü geri iade ediyorum"
text9 = "tam bir fiyat performans ürünü"
text10 = "beklediğim gibi çıkmadı"
texts = [text1, text2, text3, text4, text5, text6, text7, text8, text9, text10]

In [ ]:
tokens = tokenizer.texts_to_sequences(texts)
tokens_pad = pad_sequences(tokens, maxlen=max_tokens)
print(tokens_pad.shape)
text_pred=model.predict(tokens_pad)
print(text_pred)
text_pred = np.array([1.0 if p>0.5 else 0.0 for p in text_pred])
print(text_pred)